# IMPORT

In [1]:
from __future__ import annotations
from bqskit import Circuit, compile
from bqskit.compiler.machine import MachineModel
from bqskit.ir.gates import RXXGate, RXGate, RYGate,RZZGate,HGate,CNOTGate
from transpiler.bqskit_ionq_native_gates import GPIGate, GPI2Gate,PartialMSGate
from transpiler.qasm2_reader import load_qasm, from_qasm2_to_braket
from bqskit.ext import bqskit_to_qiskit
from qiskit_aer import AerSimulator
from qiskit import transpile
import pennylane as qml
import numpy as np
from braket.devices import LocalSimulator
from braket.circuits import Circuit as braket_Circuit
from braket.circuits import Gate


In [2]:
# Step 1: Define the MS-Gate Machine Model
IONQ_gate_set={PartialMSGate(),GPIGate(),GPI2Gate()}
def transpile_to_ms(circuit: Circuit) -> Circuit:
    """Transpile the given circuit to use only MS gates."""
    model = MachineModel(circuit.num_qudits, gate_set=IONQ_gate_set)
    compiled_circuit = compile(circuit, model, optimization_level=2)
    return compiled_circuit

# Step 2: Create a simple quantum circuit
qc = Circuit(2)
qc.append_gate(HGate(), [0])
qc.append_gate(HGate(), [1])
qc.append_gate(CNOTGate(), [1,0])
qc.append_gate(RXGate(), [1],params=[1.1561])
qc.append_gate(HGate(), [0])
qc.append_gate(RZZGate(), [0, 1],params=[1])
qc.append_gate(RYGate(), [1],params=[3])
qc.append_gate(RXXGate(), [0,1],params=[2.14584545])

    
qc_qiskit = bqskit_to_qiskit(qc)

# Step 3: Transpile the circuit to MS gates
ms_circuit = transpile_to_ms(qc)

In [3]:
#SAVE QASM
ms_circuit.save("circuit.qasm")
print(qc_qiskit.draw(output='text'))

# Initialize the latest AerSimulator
simulator = AerSimulator(method="statevector")

# Transpile the circuit for the simulator
qc_transpiled = transpile(qc_qiskit, simulator)

# Add instruction to save statevector
qc_transpiled.save_statevector()

# Run the circuit in statevector mode
result = simulator.run(qc_transpiled).result()

# Get the statevector
statevector = result.get_statevector()

# Print the statevector
np.abs(np.array(statevector))**2

     ┌───┐┌───┐    ┌───┐                      ┌──────────────┐
q_0: ┤ H ├┤ X ├────┤ H ├──────■───────────────┤0             ├
     ├───┤└─┬─┘┌───┴───┴────┐ │ZZ(1) ┌───────┐│  Rxx(2.1458) │
q_1: ┤ H ├──■──┤ Rx(1.1561) ├─■──────┤ Ry(3) ├┤1             ├
     └───┘     └────────────┘        └───────┘└──────────────┘


array([0.10533642, 0.41539816, 0.12272557, 0.35653985])

In [4]:
# Step 4: Transform to qiskit and print
qiskit_circuit = load_qasm("circuit.qasm")
print(qiskit_circuit.draw(output='text'))


     ┌──────────────┐ ┌──────────────┐┌──────────────────────────────────┐»
q_0: ┤ Gpi2(3.1416) ├─┤ Gpi2(4.7124) ├┤0                                 ├»
     ├──────────────┤┌┴──────────────┤│  Partialms(5.3779,2.3753,1.1326) │»
q_1: ┤ Gpi2(0.9739) ├┤ Gpi2(0.16904) ├┤1                                 ├»
     └──────────────┘└───────────────┘└──────────────────────────────────┘»
«     ┌─────────────┐                 ┌──────────────────────────────────┐»
«q_0: ┤ Gpi(1.7029) ├─────────────────┤0                                 ├»
«     ├─────────────┤┌───────────────┐│  Partialms(5.6235,1.9459,1.1227) │»
«q_1: ┤ Gpi(3.8102) ├┤ Gpi2(0.35887) ├┤1                                 ├»
«     └─────────────┘└───────────────┘└──────────────────────────────────┘»
«     ┌─────────────┐┌──────────────┐
«q_0: ┤ Gpi(0.1645) ├┤ Gpi2(6.2832) ├
«     ├─────────────┤├──────────────┤
«q_1: ┤ Gpi(5.4041) ├┤ Gpi2(2.4257) ├
«     └─────────────┘└──────────────┘


# NOISE

In [5]:
# Convert Qiskit circuit to PennyLane
qml_circuit = qml.from_qiskit(qiskit_circuit)
np.abs(qml.matrix(qml_circuit, wire_order=[0, 1])().T[0])**2#check

array([0.10533642, 0.12272557, 0.41539804, 0.35653997])

In [6]:
def get_noisy_counts(qiskit_qc, noise_level_two_qubit, noise_level_one_qubit, readout_error):
    # Convert the Qiskit quantum circuit into a PennyLane circuit
    qml_circuit = qml.from_qiskit(qiskit_qc)

    # Compute the adjusted noise level for single-qubit depolarizing channels in 2Q noise
    noise_level_two_qubit_single_channel = noise_level_two_qubit * 3 / 4

    # Get the number of qubits in the circuit
    number_of_qubits = qiskit_qc.num_qubits

    # Define a QNode using PennyLane's mixed-state simulator (supports noise)
    @qml.qnode(qml.device("default.mixed", wires=number_of_qubits))
    def noisy_circuit():
        # Convert the PennyLane circuit into a tape (sequence of operations)
        tape = qml.transforms.make_tape(qml_circuit)()

        # Iterate through all operations in the circuit
        for op in tape.operations:
            # Apply each gate as a unitary operation
            qml.QubitUnitary(op.parameters[0], wires=op.wires)

            # Apply **1-qubit depolarizing noise** after every 1-qubit gate
            if len(op.wires) == 1:
                qml.DepolarizingChannel(noise_level_one_qubit, wires=op.wires[0])

            # Apply **2-qubit depolarizing noise** after two-qubit (MS) gates
            if len(op.wires) == 2:
                for qubit in op.wires:
                    qml.DepolarizingChannel(noise_level_two_qubit_single_channel, wires=qubit)  # Adjusted probability

        # Apply **readout error (bit flip) before measurement**
        for qubit in range(number_of_qubits):
            qml.BitFlip(readout_error, wires=qubit)

        # Return the probability distribution over computational basis states
        return qml.probs(wires=range(number_of_qubits))

    # Run the noisy circuit simulation and return the result
    noisy_result = noisy_circuit()
    return noisy_result
print("Noisy     simulation result:", get_noisy_counts(qiskit_circuit, noise_level_two_qubit=0.02, noise_level_one_qubit=0.001, readout_error=0.05))

print("Noiseless simulation result:",np.abs(qml.matrix(qml_circuit, wire_order=[0, 1])().T[0])**2)#check

Noisy     simulation result: [0.13093159 0.13557134 0.38913469 0.34436238]
Noiseless simulation result: [0.10533642 0.12272557 0.41539804 0.35653997]


# Vibratium

In [7]:
bracket_qc=from_qasm2_to_braket("circuit.qasm")


In [8]:
print(bracket_qc)

T  : │        0        │      1       │      2       │           3            │      4      │      5       │           6            │      7      │      8       │       9       │
                        ┌────────────┐ ┌────────────┐ ┌──────────────────────┐ ┌───────────┐                ┌──────────────────────┐ ┌───────────┐ ┌────────────┐                 
q0 : ───StartVerbatim───┤ GPi2(3.14) ├─┤ GPi2(4.71) ├─┤ MS(5.38, 2.38, 1.13) ├─┤ GPi(1.70) ├────────────────┤ MS(5.62, 1.95, 1.12) ├─┤ GPi(0.16) ├─┤ GPi2(6.28) ├───EndVerbatim───
              ║         └────────────┘ └────────────┘ └──────────┬───────────┘ └───────────┘                └──────────┬───────────┘ └───────────┘ └────────────┘        ║        
              ║         ┌────────────┐ ┌────────────┐ ┌──────────┴───────────┐ ┌───────────┐ ┌────────────┐ ┌──────────┴───────────┐ ┌───────────┐ ┌────────────┐        ║        
q1 : ─────────╨─────────┤ GPi2(0.97) ├─┤ GPi2(0.17) ├─┤ MS(5.38, 2.38, 1.13) ├─┤ GPi(3.81) ├─┤ GPi2(0.36)

In [9]:
device = LocalSimulator() # Choose the local simulator
shots = 100000  # Number of times to execute the circuit

result = device.run(bracket_qc, shots=shots).result()
counts = result.measurement_probabilities  # Get the shot results

print(counts)  # Dictionary {bitstring: counts}
print("Noiseless simulation result:",np.abs(qml.matrix(qml_circuit, wire_order=[0, 1])().T[0])**2)#check

{'11': 0.35699, '10': 0.41282, '00': 0.10653, '01': 0.12366}
Noiseless simulation result: [0.10533642 0.12272557 0.41539804 0.35653997]


# GATE braket TEST

In [10]:
print(braket_Circuit().gpi(0,1))

T  : │    0     │
      ┌────────┐ 
q0 : ─┤ GPi(1) ├─
      └────────┘ 
T  : │    0     │


In [43]:
braket_Circuit().gpi(0,0.445).to_unitary()

array([[0.        +0.j        , 0.90261067-0.43045788j],
       [0.90261067+0.43045788j, 0.        +0.j        ]])

In [44]:
phi=0.445
np.array([
            [0, np.exp(-1j * phi)],
            [np.exp(1j * phi), 0]
        ])

array([[0.        +0.j        , 0.90261067-0.43045788j],
       [0.90261067+0.43045788j, 0.        +0.j        ]])

In [46]:
braket_Circuit().gpi2(0,0.445).to_unitary()

array([[ 0.70710678+0.j        , -0.30437969-0.63824212j],
       [ 0.30437969-0.63824212j,  0.70710678+0.j        ]])

In [45]:
 # Extract the phase parameter
factor = 1 / np.sqrt(2)  # Normalization factor
exp_pos = -1j * np.exp(1j * phi)
exp_neg = -1j * np.exp(-1j * phi)

factor * np.array([
            [1, exp_neg],
            [exp_pos, 1]
        ])

array([[ 0.70710678+0.j        , -0.30437969-0.63824212j],
       [ 0.30437969-0.63824212j,  0.70710678+0.j        ]])

In [61]:
phi1=0.2455
phi2=0.4157
theta=0.11545
q1=0
q2=1
braket_Circuit().ms(q1, q2, phi1, phi2,theta).to_unitary()

array([[ 0.99833437+0.j        ,  0.        +0.j        ,
         0.        +0.j        , -0.03542718-0.0455345j ],
       [ 0.        +0.j        ,  0.99833437+0.j        ,
         0.009772  -0.05685934j,  0.        +0.j        ],
       [ 0.        +0.j        , -0.009772  -0.05685934j,
         0.99833437+0.j        ,  0.        +0.j        ],
       [ 0.03542718-0.0455345j ,  0.        +0.j        ,
         0.        +0.j        ,  0.99833437+0.j        ]])

In [62]:
phi0=phi1
phi1=phi2

cos_theta = np.cos(theta/2)
sin_theta = np.sin(theta/2)

e_pos = -1j * np.exp(1j * (phi0 + phi1))
e_neg = -1j * np.exp(-1j * (phi0 + phi1))
e_diff_pos = -1j * np.exp(1j * (phi0 - phi1))
e_diff_neg = -1j * np.exp(-1j * (phi0 - phi1))

np.array([
    [cos_theta, 0, 0, e_neg * sin_theta],
    [0, cos_theta, e_diff_neg * sin_theta, 0],
    [0, e_diff_pos * sin_theta, cos_theta, 0],
    [e_pos * sin_theta, 0, 0, cos_theta]
])

array([[ 0.99833437+0.j        ,  0.        +0.j        ,
         0.        +0.j        , -0.03542718-0.0455345j ],
       [ 0.        +0.j        ,  0.99833437+0.j        ,
         0.009772  -0.05685934j,  0.        +0.j        ],
       [ 0.        +0.j        , -0.009772  -0.05685934j,
         0.99833437+0.j        ,  0.        +0.j        ],
       [ 0.03542718-0.0455345j ,  0.        +0.j        ,
         0.        +0.j        ,  0.99833437+0.j        ]])